Imports

In [2]:
import os
import glob
import ee
import geemap
import rasterio
import numpy as np
import matplotlib.pyplot as plt

Initialized GEE

In [3]:
ee.Authenticate()
ee.Initialize(project='wildfire-lifecycle')

File Paths

In [4]:
raw_dir = "../data/raw"
os.makedirs(raw_dir, exist_ok=True)

Define Fire Events

In [5]:
fires = {
    "dixie_fire": {
        # Dixie Fire AOI (approx perimeter)
        "aoi": ee.Geometry.Polygon(
            [[[-121.6, 39.9], [-120.6, 39.9], [-120.6, 40.5], [-121.6, 40.5], [-121.6, 39.9]]]
        ),
        "start": "2021-07-01",
        "end": "2021-10-01"
    },
    "camp_fire": {
        # Camp Fire AOI (Butte County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-121.75, 39.6], [-121.3, 39.6], [-121.3, 40.1], [-121.75, 40.1], [-121.75, 39.6]]]
        ),
        "start": "2018-11-01",
        "end": "2018-12-15"
    },
    "bootleg_fire": {
        # Bootleg Fire AOI (Lake County, OR)
        "aoi": ee.Geometry.Polygon(
            [[[-121.2, 42.3], [-120.2, 42.3], [-120.2, 42.8], [-121.2, 42.8], [-121.2, 42.3]]]
        ),
        "start": "2021-07-01",
        "end": "2021-08-31"
    },
    "caldor_fire": {
        # Caldor Fire AOI (Eldorado County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-120.8, 38.6], [-120.2, 38.6], [-120.2, 39.1], [-120.8, 39.1], [-120.8, 38.6]]]
        ),
        "start": "2021-08-01",
        "end": "2021-10-01"
    },
    "east_troublesome_fire": {
        # East Troublesome Fire AOI (Grand County, CO)
        "aoi": ee.Geometry.Polygon(
            [[[-106.2, 40.0], [-105.6, 40.0], [-105.6, 40.4], [-106.2, 40.4], [-106.2, 40.0]]]
        ),
        "start": "2020-10-01",
        "end": "2020-11-15"
    },
    "carr_fire": {
        # Carr Fire AOI (Shasta County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-122.6, 40.4], [-122.0, 40.4], [-122.0, 40.8], [-122.6, 40.8], [-122.6, 40.4]]]
        ),
        "start": "2018-07-23",
        "end": "2018-09-04"
    },
    "zogg_fire": { 
        # Zogg Fire AOI (Shasta County, CA) 
        "aoi": ee.Geometry.Polygon( 
            [[[-122.8, 40.4], [-122.3, 40.4], [-122.3, 39.9], [-122.8, 39.9], [-122.8, 40.4]]] 
        ),
        "start": "2020-09-27",
        "end": "2020-10-13" },
    "red_salmon_fire": { 
        # Red Salmon Complex AOI (Humboldt/Trinity County, CA) 
        "aoi": ee.Geometry.Polygon(
            [[[-123.8, 41.0], [-123.4, 41.0], [-123.4, 41.4], [-123.8, 41.4], [-123.8, 41.0]]] 
        ), 
        "start": "2020-07-27", 
        "end": "2020-10-15" },
    "glass_fire": {
        # Glass Fire AOI (Napa/Sonoma County, CA)
        "aoi": ee.Geometry.Polygon(
            [[[-122.6, 38.4], [-122.3, 38.4], [-122.3, 38.7], [-122.6, 38.7], [-122.6, 38.4]]]
        ),
        "start": "2020-09-27",
        "end": "2020-10-20"
    }
}


Helper Functions

In [7]:
def compute_ndvi(img):
    return img.normalizedDifference(['B8', 'B4']).rename("NDVI")

def compute_nbr(img):
    return img.normalizedDifference(['B8', 'B12']).rename("NBR")

def export_image(image, out_path, region, scale):
    try:
        geemap.ee_export_image(
            image,
            filename=out_path,
            scale=scale,
            region=region,
            file_per_band=False
        )
        print(f"Saved: {out_path}")
    except Exception as e:
        print(f"  ERROR exporting {out_path}: {e}")

Main Extraction Loop

In [ ]:
for fire_name, fire_info in fires.items():
    print(f"Processing {fire_name}...")
    aoi = fire_info["aoi"]
    start, end = fire_info["start"], fire_info["end"]
    fire_dir = os.path.join(raw_dir, fire_name)
    os.makedirs(fire_dir, exist_ok=True)

    # Pre-fire (30 days before start)
    pre_start = ee.Date(start).advance(-30, "day")
    pre_end   = ee.Date(start)

    # Post-fire (30 days after start)
    post_start = ee.Date(end)
    post_end   = ee.Date(end).advance(30, "day")

    # MODIS Burned Area
    burn = (ee.ImageCollection("MODIS/061/MCD64A1")
            .filterDate(start, end)
            .filterBounds(aoi)
            .select("BurnDate")
            .max()
            .clip(aoi))

    export_image(burn, f"{fire_dir}/modis_burned_area.tif", aoi, scale=500)

    # Sentinel-2 NDVI and NBR
    s2_pre = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterDate(pre_start, pre_end)
              .filterBounds(aoi))

    s2_post = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
               .filterDate(post_start, post_end)
               .filterBounds(aoi))

    pre_ndvi = s2_pre.map(compute_ndvi).mean().clip(aoi)
    post_ndvi = s2_post.map(compute_ndvi).mean().clip(aoi)

    pre_nbr = s2_pre.map(compute_nbr).mean().clip(aoi)
    post_nbr = s2_post.map(compute_nbr).mean().clip(aoi)

    export_image(pre_ndvi, f"{fire_dir}/pre_ndvi.tif", aoi, scale=30)
    export_image(post_ndvi, f"{fire_dir}/post_ndvi.tif", aoi, scale=30)
    export_image(pre_nbr, f"{fire_dir}/pre_nbr.tif", aoi, scale=30)
    export_image(post_nbr, f"{fire_dir}/post_nbr.tif", aoi, scale=30)

    # Landcover
    lc = ee.Image("ESA/WorldCover/v200/2021").clip(aoi)
    export_image(lc, f"{fire_dir}/landcover.tif", aoi, scale=30)

    # ERA5 Temperature
    era_temp = (ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
                .filterDate(start, end)
                .select("temperature_2m")
                .mean()
                .clip(aoi))

    export_image(era_temp, f"{fire_dir}/era5_temp.tif", aoi, scale=1000)

    # CHIRPS Precip
    precip = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
              .filterDate(start, end)
              .select("precipitation")
              .sum()
              .clip(aoi))

    export_image(precip, f"{fire_dir}/chirps_precip.tif", aoi, scale=5000)

    # SRTM DEM
    dem = ee.Image("USGS/SRTMGL1_003").clip(aoi)
    export_image(dem, f"{fire_dir}/srtm_dem.tif", aoi, scale=30)

Processing dixie_fire...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_fire\modis_burned_area.tif
Saved: ../data/raw\dixie_fire/modis_burned_area.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_fire\pre_ndvi.tif
Saved: ../data/raw\dixie_fire/pre_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_fire\post_ndvi.tif
Saved: ../data/raw\dixie_fire/post_ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_fire\pre_nbr.tif
Saved: ../data/raw\dixie_fire/pre_nbr.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_fire\post_nbr.tif
Saved: ../data/raw\dixie_fire/post_nbr.tif
Generating U